In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, classification_report
from imblearn.over_sampling import SMOTE

# Load data
data_path = r"C:\Job_og_eksamensbevis\Github\projekter\RF_project\data\creditcard_preprocessed.csv"
df = pd.read_csv(data_path)

X = df.drop('Class', axis=1)
y = df['Class']

# Split data i træning+validering og test (20% test, stratificeret)
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42)

# Stratified K-Fold på træning+validering (80%)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

precisions = []
recalls = []
f1s = []

for fold, (train_idx, val_idx) in enumerate(skf.split(X_train_val, y_train_val), 1):
    X_train, X_val = X_train_val.iloc[train_idx], X_train_val.iloc[val_idx]
    y_train, y_val = y_train_val.iloc[train_idx], y_train_val.iloc[val_idx]

    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

    rf = RandomForestClassifier(
        n_estimators=100,
        random_state=42,
        n_jobs=-1
    )

    rf.fit(X_train_resampled, y_train_resampled)
    y_val_pred = rf.predict(X_val)

    precision = precision_score(y_val, y_val_pred)
    recall = recall_score(y_val, y_val_pred)
    f1 = f1_score(y_val, y_val_pred)

    precisions.append(precision)
    recalls.append(recall)
    f1s.append(f1)

    print(f"\nFold {fold}:")
    print("Confusion Matrix:")
    print(confusion_matrix(y_val, y_val_pred))
    print(f"Precision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"F1-score:  {f1:.4f}")
    print("Classification report:")
    print(classification_report(y_val, y_val_pred))

print("\n=== Gennemsnitlig performance over 5 folds ===")
print(f"Gns. Precision: {sum(precisions)/len(precisions):.4f}")
print(f"Gns. Recall:    {sum(recalls)/len(recalls):.4f}")
print(f"Gns. F1-score:  {sum(f1s)/len(f1s):.4f}")

# --- Endelig test på hold-out test-sæt ---

# Træn model på hele træning+valideringsdata (med SMOTE)
smote = SMOTE(random_state=42)
X_train_val_resampled, y_train_val_resampled = smote.fit_resample(X_train_val, y_train_val)

final_rf = RandomForestClassifier(n_estimators=100,
                                  random_state=42,
                                  n_jobs=-1
)

final_rf.fit(X_train_val_resampled, y_train_val_resampled)

# Forudsig på det uafhængige test-sæt
y_test_pred = final_rf.predict(X_test)

print("\n=== Endelig evaluering på test-sæt ===")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_test_pred))
print(f"Precision: {precision_score(y_test, y_test_pred):.4f}")
print(f"Recall:    {recall_score(y_test, y_test_pred):.4f}")
print(f"F1-score:  {f1_score(y_test, y_test_pred):.4f}")
print("Classification report:")
print(classification_report(y_test, y_test_pred))

import joblib


# Gem den trænede model
joblib.dump(final_rf, "rf_model.pkl")

